# EEG DataLoader

The dimensions of the training set are as follows: 4,500 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_train.

The dimensions of the testing set are as follows: 750 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_test.

You can download it from this Google Drive link: [https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing]. 
Please download the data and place it in your data_path at "./data."

Installing needed packages

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

This is so that you do not have to repeat ./data/ for every single call to the data

In [2]:
data_path = './data/'

This loads in the data, normalizes the data then converts them to PyTorch tensors

In [3]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

train_data_mean = np.mean(train_data)
train_data_std = np.std(train_data)
train_data_normalized = (train_data - train_data_mean) / train_data_std
test_data_normalized = (test_data - train_data_mean) / train_data_std

#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data_normalized)
y_train_tensor = torch.LongTensor(train_label)
x_test_tensor = torch.Tensor(test_data_normalized)
y_test_tensor = torch.LongTensor(test_label)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

In [5]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

# Build simple Deep learning model

This creates the model

In [6]:
class EEGAutoencoderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EEGAutoencoderClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 795, 384), # Input dimention is 64 channel * 795 time point, and use 256 units for first NN layer
            nn.BatchNorm1d(384),
            nn.LeakyReLU(negative_slope=0.05), # Use LeakyReLu function for NN training 
            nn.Dropout(0.05),
            nn.Linear(384, 256), # 256 NN units to 192 units
            nn.BatchNorm1d(256),
            nn.LeakyReLU(negative_slope=0.05),
            nn.Dropout(0.05),
            nn.Linear(256, 192),#  192 NN units to 128 units
            nn.LeakyReLU(negative_slope=0.05)
        )
        self.classifier = nn.Sequential(
            nn.Linear(192, num_classes), # num_classes is 5 (hello,” “help me,” “stop,” “thank you,” and “yes”)
            nn.LogSoftmax(dim=1)  # Use LogSoftmax for multi-class classification
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)
        
        # import pdb;pdb.set_trace()
        x = self.classifier(x)
        return x

This sets all the parameters needed for the training.

In [7]:
num_classes = 5 # setting final output class
model = EEGAutoencoderClassifier(num_classes).to(device) 
criterion = nn.NLLLoss() # Use NLLLoss function to optimize
optimizer = optim.Adam(model.parameters(), lr=0.001) # Setting parameters learning rate = 0.001

This will train the model. Changing the number of epochs and other hyper parameters will change the amount of time that it needs to run and changes the accuracy.

In [8]:
num_epochs = 50 # setting training epochs (Number of training iterations)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader: 
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 1.6468422412872314
Epoch 2/50, Loss: 1.5850285291671753
Epoch 3/50, Loss: 1.5205365419387817
Epoch 4/50, Loss: 1.532263994216919
Epoch 5/50, Loss: 1.3678032159805298
Epoch 6/50, Loss: 1.2575147151947021
Epoch 7/50, Loss: 1.0292750597000122
Epoch 8/50, Loss: 1.2314276695251465
Epoch 9/50, Loss: 0.9233094453811646
Epoch 10/50, Loss: 1.1760468482971191
Epoch 11/50, Loss: 0.6848042011260986
Epoch 12/50, Loss: 0.5860562324523926
Epoch 13/50, Loss: 0.7431285381317139
Epoch 14/50, Loss: 0.48889076709747314
Epoch 15/50, Loss: 0.6034378409385681
Epoch 16/50, Loss: 0.3780413269996643
Epoch 17/50, Loss: 0.2878186106681824
Epoch 18/50, Loss: 0.2521151602268219
Epoch 19/50, Loss: 0.4437309205532074
Epoch 20/50, Loss: 0.3797251880168915
Epoch 21/50, Loss: 0.25286126136779785
Epoch 22/50, Loss: 0.16965611279010773
Epoch 23/50, Loss: 0.24978816509246826
Epoch 24/50, Loss: 0.24163788557052612
Epoch 25/50, Loss: 0.12577393651008606
Epoch 26/50, Loss: 0.22460056841373444
Epoch 27/50, Lo

Evaluates the model and gives a testing accuracy

In [9]:
model.eval() # Evaluate your model
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 51.70%
